# Accuracy Assessment

In [1]:
import geopandas as gpd
from rasterstats import zonal_stats
import numpy as np
import matplotlib.pyplot as plt
import os
from shapely.ops import unary_union
import pandas as pd

In [2]:
def total_length_within_polygons(trails, polygons):
    length_within_polygon = []
    for polygon in polygons.geometry:
        trails_within_polygon = trails[trails.geometry.intersects(polygon)]
        total_length = int(trails_within_polygon.length.sum())
        length_within_polygon.append(total_length)
    return length_within_polygon

def calculate_covered_area(lengths, width):
    # Calculate the area covered by the trails in square meters
    return [length * width for length in lengths]

# Function to categorize each row
def categorize(row):
    if row['Is_TP']:
        return 'TP'
    elif row['Is_FP']:
        return 'FP'
    elif row['Is_FN']:
        return 'FN'
    elif row['Is_TN']:
        return 'TN'
    else:
        return 'Unknown'

In [ ]:
'''The code first reads the weak, LiDAR, and interpolated trail datasets, merges them into a single GeoDataFrame, 
and saves this as a new file.'''

### Reasing strong, weak, lidar, and interpolated trails
weak_trails = gpd.read_file('/media/irro/Irro/HumanFootprint/AA_delineation/XC_trails_weak.gpkg')
lidar_trails = gpd.read_file('/media/irro/Irro/HumanFootprint/AA_delineation/XC_trails_LiDAR.gpkg')
interpolated_trails = gpd.read_file('/media/irro/Irro/HumanFootprint/AA_delineation/XC_trails_interpolated.gpkg')
strong_trails = gpd.read_file('/media/irro/Irro/HumanFootprint/AA_delineation/XC_trails_strong.gpkg')
ira_trails = gpd.read_file('/media/irro/Irro/HumanFootprint/AA_delineation/Ira_trails.shp')

#### AOIs
polygons = gpd.read_file('/media/irro/Irro/HumanFootprint/AA_delineation/polygons.shp')

merged_weak_trails = gpd.GeoDataFrame(pd.concat([weak_trails, lidar_trails, interpolated_trails], ignore_index=True))
merged_weak_trails.to_file("/media/irro/Irro/HumanFootprint/AA_delineation/merged_weak_trails.gpkg", driver="GPKG")

ira_weak_trails = ira_trails[ira_trails['id'].isin([2, 4, 3]) | ira_trails['id'].isna()]

all_weak_trails = gpd.GeoDataFrame(pd.concat([ira_weak_trails, merged_weak_trails], ignore_index=True))
all_weak_trails = all_weak_trails[['geometry', 'description']]
all_weak_trails['description'] = 'weak trails'
all_weak_trails = all_weak_trails.set_crs(epsg=2956, allow_override=True)

all_weak_trails.to_file("/media/irro/Irro/HumanFootprint/AA_delineation/all_weak_trails.gpkg", driver="GPKG")

In [3]:
# Load the data
polygons = gpd.read_file('/media/irro/Irro/HumanFootprint/AA_delineation/polygons.shp')
all_trails = gpd.read_file("/media/irro/Irro/HumanFootprint/AA/all_trails.shp")
seismic_lines = gpd.read_file('/media/irro/Irro/Kirby/FootprintsKirby/Footprint_SiteE.shp')
buffered_trails = gpd.read_file('/media/irro/Irro/HumanFootprint/AA/all_trails_0.5buf.shp')
cnn_map = '/media/irro/Irro/HumanFootprint/cnn_map_test.tif'

# Set CRS for all_trails to match polygons if it's None
if all_trails.crs is None:
    all_trails = all_trails.set_crs(polygons.crs)
    print("CRS set for all_trails to match polygons.")

# Reproject both polygons and all_trails to match seismic_lines CRS
polygons = polygons.to_crs(seismic_lines.crs)
all_trails = all_trails.to_crs(seismic_lines.crs)
buffered_trails = buffered_trails.to_crs(polygons.crs)

# Check CRS
print("CRS for polygons:", polygons.crs)
print("CRS for all_trails:", all_trails.crs)
print("CRS for seismic_lines:", seismic_lines.crs)

CRS set for all_trails to match polygons.
CRS for polygons: epsg:2956
CRS for all_trails: epsg:2956
CRS for seismic_lines: epsg:2956


### Total Length / Area in AOI

In [ ]:
'''For each polygon in your areas for accuracy assessment, it calculates the total length of trails 
(both weak and strong) that intersect with that polygon. The total lengths are added to the polygons 
GeoDataFrame and saved as a new file.'''

polygons = gpd.read_file('/media/irro/Irro/HumanFootprint/AA_delineation/polygons.shp')
polygons = polygons.to_crs(seismic_lines.crs)

# Calculate lengths
length_weak = total_length_within_polygons(all_weak_trails, polygons)
length_strong = total_length_within_polygons(strong_trails, polygons)
length_total = total_length_within_polygons(pd.concat([strong_trails, all_weak_trails], ignore_index=True), polygons)

# Add the lengths to the polygons DataFrame
polygons['length_m_weak'] = length_weak
polygons['length_m_strong'] = length_strong
polygons['total_m_length'] = length_total

######## Calculate lengths per hectare

polygons['area_ha'] = polygons.area / 10000
polygons['length_km_weak_per_ha'] = [int(lw / a) if a != 0 else 0 for lw, a in zip(length_weak, polygons['area_ha'])]
polygons['length_km_strong_per_ha'] = [int(ls / a) if a != 0 else 0 for ls, a in zip(length_strong, polygons['area_ha'])]
polygons['total_km_length_per_ha'] = [int(lt / a) if a != 0 else 0 for lt, a in zip(length_total, polygons['area_ha'])]

######## Calculate lengths in kilometers per km2
polygons['area_km2'] = polygons.area / 1000000
polygons['length_km_weak_per_km2'] = [int(lw / a / 1000) if a != 0 else 0 for lw, a in zip(length_weak, polygons['area_km2'])]
polygons['length_km_strong_per_km2'] = [int(ls / a / 1000) if a != 0 else 0 for ls, a in zip(length_strong, polygons['area_km2'])]
polygons['total_km_length_per_km2'] = [int(lt / a / 1000) if a != 0 else 0 for lt, a in zip(length_total, polygons['area_km2'])]
polygons['fid'] = polygons['fid'].astype(int)

# Output the results
out_poly = "/media/irro/Irro/HumanFootprint/AA_delineation/polygons_with_trail_lengths_new.gpkg"
polygons.to_file(out_poly, driver="GPKG")

### Calculate area (VI) of buffered trails

In [ ]:
# Width of the trail in meters
trail_width_m = 0.3

# Calculate the area covered by each type of trail in square meters
area_covered_by_weak_trails = calculate_covered_area(length_weak, trail_width_m)
area_covered_by_strong_trails = calculate_covered_area(length_strong, trail_width_m)
area_covered_by_all_trails = calculate_covered_area(length_total, trail_width_m)
polygons['area_weak'] = area_covered_by_weak_trails
polygons['area_strong'] = area_covered_by_strong_trails
polygons['area_alltrails'] = area_covered_by_all_trails

# Calculate the percentage of area covered
polygons['percent_area_weak_ha'] = [(aw / a) * 100 if a != 0 else 0 for aw, a in zip(area_covered_by_weak_trails, polygons['area_ha'] * 10000)]
polygons['percent_area_strong_ha'] = [(as_ / a) * 100 if a != 0 else 0 for as_, a in zip(area_covered_by_strong_trails, polygons['area_ha'] * 10000)]
polygons['percent_area_total_ha'] = [(at / a) * 100 if a != 0 else 0 for at, a in zip(area_covered_by_all_trails, polygons['area_ha'] * 10000)]

polygons['percent_area_weak_km2'] = [(aw / a) * 100 if a != 0 else 0 for aw, a in zip(area_covered_by_weak_trails, polygons['area_km2'] * 1000000)]
polygons['percent_area_strong_km2'] = [(as_ / a) * 100 if a != 0 else 0 for as_, a in zip(area_covered_by_strong_trails, polygons['area_km2'] * 1000000)]
polygons['percent_area_total_km2'] = [(at / a) * 100 if a != 0 else 0 for at, a in zip(area_covered_by_all_trails, polygons['area_km2'] * 1000000)]

# Output the results
polygons.to_file(out_poly, driver="GPKG")

In [ ]:
polygons= gpd.read_file(out_poly)
polygons[poly.columns[0:-1]]

### Mean area of trails

In [ ]:
mean_area_weak = int(polygons.area_weak.mean())
std_area_weak = int(polygons.area_weak.std())

print('Mean area of weak trails:\n', f"{mean_area_weak} ± {std_area_weak}")
print('% Mean area of weak trails:\n', f"{int(mean_area_weak/25)} ± {int(std_area_weak/25)}")

In [ ]:
mean_area_strong = int(polygons.area_strong.mean())
std_area_strong = int(polygons.area_strong.std())

print('Mean area of strong trails:\n', f"{mean_area_strong} ± {std_area_strong}")
print('% Mean area of strong trails:\n', f"{int(mean_area_strong/25)} ± {int(std_area_strong/25)}")

In [ ]:
mean_area_all = int(polygons.area_alltrails.mean())
std_area_all = int(polygons.area_alltrails.std())

print('Mean area of ALL trails:\n', f"{mean_area_all} ± {std_area_all}")
print('% Mean area of ALL trails:\n', f"{int(mean_area_all/25)} ± {int(std_area_all/25)}")

### Mean length of trails

In [ ]:
mean_length_weak = int(polygons.length_m_weak.mean())
std_length_weak = int(polygons.length_m_weak.std())

print('Mean length of weak trails:\n', f"{mean_length_weak} ± {std_length_weak}")

In [ ]:
mean_length_strong = int(polygons.length_m_strong.mean())
std_length_strong = int(polygons.length_m_strong.std())

print('Mean length of strong trails:\n', f"{mean_length_strong} ± {std_length_strong}")

##### Mean length per km2

In [ ]:
mean_length = int(polygons.total_km_length_per_km2.mean())
std_length = int(polygons.total_km_length_per_km2.std())

print('Mean length of all trails:\n', f"{mean_length} ± {std_length} per km2")

In [ ]:
mean_length = int(polygons.length_km_strong_per_km2.mean())
std_length = int(polygons.length_km_strong_per_km2.std())

print('Mean length of STRONG trails:\n', f"{mean_length} ± {std_length} per km2")

### DTM: Trail Depths

In [ ]:
import rasterio
import geopandas as gpd
import numpy as np
from rasterio.mask import mask

def get_stats(values):
    """ Calculate median and IQR, excluding NaN values """
    valid_values = values[~np.isnan(values)]*100
    if valid_values.size > 0:
        median = int(np.median(valid_values))
        iqr75 = int(np.percentile(valid_values, 75))
        iqr25 = int(np.percentile(valid_values, 25))
        return median, iqr25, iqr75
    else:
        return np.nan, np.nan


# Load the polygons and trails data
polygons = gpd.read_file('/media/irro/Irro/HumanFootprint/RandomPolies_Kirby_trails_visint.gpkg')

trails = gpd.read_file('/home/irro/Downloads/XC_trails_strong.gpkg')  # Example file path

# Open the normalized DTM raster
with rasterio.open('/media/irro/Irro/HumanFootprint/Kirby_DTM50cm_norm_2022.vrt') as src:

    for index, polygon in polygons.iterrows():
        # Mask the DTM with the current polygon to extract values
        out_image, out_transform = mask(src, [polygon['geometry']], crop=True)
        dtm_values = np.array(out_image).flatten()  # Convert to numpy array and then flatten
        
        # Mask the DTM with the trail geometry to extract trail values
        # Assuming that 'trails' is a GeoDataFrame with trail geometries
        trail_geom = trails[trails.geometry.intersects(polygon['geometry'])]
        if not trail_geom.empty:
            trail_image, _ = mask(src, [trail_geom['geometry'].iloc[0]], crop=True)
            trail_values = np.array(trail_image).flatten()

            # Exclude no-data values from calculations
            valid_trail_values = trail_values[trail_values != src.nodata]
            valid_dtm_values = dtm_values[dtm_values != src.nodata]
            valid_hollow_values = dtm_values[(dtm_values != src.nodata) & (dtm_values < 0.2)]
            
            if valid_trail_values.size > -2:
                # Calculate statistics for trails
                median_trail, tr_iqr25, tr_iqr75 = get_stats(valid_trail_values)
                non_trail_values = np.setdiff1d(valid_dtm_values, valid_trail_values, assume_unique=True)
                median_non_trail, nt_iqr25, nt_iqr75 = get_stats(non_trail_values)
                
                # Add the statistics to the polygons DataFrame
                polygons.at[index, 'iqr25_dtm_trail'] = tr_iqr25
                polygons.at[index, 'median_dtm_trail'] = median_trail
                polygons.at[index, 'iqr75_dtm_trail'] = tr_iqr75
                polygons.at[index, 'iqr25_dtm_non_trail'] = nt_iqr25
                polygons.at[index, 'median_dtm_non_trail'] = median_non_trail
                polygons.at[index, 'iqr75_dtm_non_trail'] = nt_iqr75
                polygons.at[index, 'dif'] = median_non_trail-median_trail

                # Optionally, you can enforce integer type for the entire DataFrame or specific columns
polygons = polygons.astype({'iqr25_dtm_trail': 'Int64', 'median_dtm_trail': 'Int64', 'iqr75_dtm_trail': 'Int64', 
                            'iqr25_dtm_non_trail': 'Int64', 'median_dtm_non_trail': 'Int64', 'iqr75_dtm_non_trail': 'Int64', 
                            'dif': 'Int64'}, errors='ignore')

# Save the updated polygons DataFrame
out_poly = "/media/irro/Irro/HumanFootprint/AA_delineation/trails_DTMdepths.gpkg"
polygons.to_file(out_poly, driver="GPKG")

In [ ]:
polygons[polygons.columns[3:]]

In [ ]:
# Calculating mean and standard deviation for area coverage percentage of strong trails per square kilometer
mean_dif = int(np.mean(polygons['dif']))
std_dif = int(np.std(polygons['dif']))

print("Mean difference in normDTM between trails and non-trails:", f"{mean_dif} ± {std_dif} cm")

### Length of VI trails on lines

##### Pnly seicmic line areas

In [ ]:


# # DataFrame to store the results
# results_list = []

# for index, polygon in polygons.iterrows():
#     # Intersect seismic lines with the polygon
#     seismic_in_polygon = gpd.overlay(seismic_lines, gpd.GeoDataFrame([polygon], columns=['geometry'], crs=polygons.crs), how='intersection')

#     # Calculate the area of seismic lines within the polygon
#     seismic_area_in_polygon = seismic_in_polygon.geometry.area.sum()
#     print(seismic_area_in_polygon)
#     # Intersect buffered trails with seismic lines within the polygon
#     trails_within_seismic = gpd.overlay(buffered_trails, seismic_in_polygon, how='intersection')

#     # Calculate the area of buffered trails within the seismic lines in the polygon
#     trails_area_within_seismic = trails_within_seismic.geometry.area.sum()
#     print(trails_area_within_seismic)
#     # Store the results
#     results_list.append({
#         'Polygon_ID': index,
#         'Seismic_Area_in_Polygon': seismic_area_in_polygon,
#         'Trails_Area_within_Seismic': trails_area_within_seismic
#     })

# # Convert to DataFrame
# lines = pd.DataFrame(results_list)

# # Optionally, save to a CSV file
# results_csv_path = '/media/irro/Irro/HumanFootprint/AA/line_areas.csv'
# lines.to_csv(results_csv_path, index=False)

# # Print the DataFrame
# print(lines)

In [ ]:

# Load the data
polygons = gpd.read_file('/media/irro/Irro/HumanFootprint/AA_delineation/polygons.shp')
all_trails = gpd.read_file("/media/irro/Irro/HumanFootprint/AA/all_trails.shp")
seismic_lines = gpd.read_file('/media/irro/Irro/Kirby/FootprintsKirby/Footprint_SiteE.shp')
buffered_trails = gpd.read_file('/media/irro/Irro/HumanFootprint/AA/all_trails_0.5buf.shp')
cnn_map = '/media/irro/Irro/HumanFootprint/cnn_map_test.tif'

# Set CRS for all_trails to match polygons if it's None
if all_trails.crs is None:
    all_trails = all_trails.set_crs(polygons.crs)
    print("CRS set for all_trails to match polygons.")

# Reproject both polygons and all_trails to match seismic_lines CRS
polygons = polygons.to_crs(seismic_lines.crs)
all_trails = all_trails.to_crs(seismic_lines.crs)
buffered_trails = buffered_trails.to_crs(polygons.crs)

# Check CRS
print("CRS for polygons:", polygons.crs)
print("CRS for all_trails:", all_trails.crs)
print("CRS for seismic_lines:", seismic_lines.crs)

In this script, for each polygon, you are calculating:

The total area of seismic lines within the polygon.
The total area of buffered VI trails within the seismic lines in the polygon.
The total area of predicted trails within the seismic lines in the polygon.
The percentage of these trail areas relative to the total seismic area within the polygon.

In [ ]:
from rasterio.features import geometry_mask
from rasterio.mask import mask as rio_mask

# DataFrame to store the results
results_list = []

# Open the raster file for trail predictions
with rasterio.open(cnn_map) as src:
    affine = src.transform

    # Loop through each polygon
    for index, polygon in polygons.iterrows():
        # Intersect seismic lines with the current polygon to find the part of seismic lines within it
        seismic_in_polygon = gpd.overlay(seismic_lines, gpd.GeoDataFrame([polygon], columns=['geometry'], crs=polygons.crs), how='intersection')
        
        # Check if there are seismic lines within the polygon
        if not seismic_in_polygon.empty:
            # Calculate the total area of the seismic lines within the polygon
            seismic_area = seismic_in_polygon.geometry.unary_union.area

            # Intersect buffered trails with the seismic lines within the polygon
            trails_within_seismic = gpd.overlay(buffered_trails, seismic_in_polygon, how='intersection')
            
            # Calculate the total area covered by the buffered VI trails within the seismic lines in the polygon
            trails_area_within_seismic = trails_within_seismic.geometry.area.sum()

            # If seismic area is more than zero, calculate the area of predicted trails within this area
            if seismic_area > 0:
                seismic_geom = seismic_in_polygon.unary_union
                intersected_area = polygon['geometry'].intersection(seismic_geom)

                # Create a mask for the intersected area
                intersected_mask = geometry_mask([intersected_area], transform=src.transform, invert=True, out_shape=src.shape)

                # Extract the masked area from the raster for predicted trails
                masked_raster, _ = rio_mask(src, [intersected_area], crop=True)
                trail_raster = np.where(masked_raster[0] > 10, 1, 0)  # Apply threshold for trail detection

                # Calculate the total area of predicted trails within the intersected area
                trail_area_pixels = trail_raster.sum()
                trail_area_meters = trail_area_pixels * (src.res[0] * src.res[1])

                # Calculate the percentage of VI and predicted trails area relative to the total seismic footprint area
                percent_vi = (trails_area_within_seismic / seismic_area) * 100 if seismic_area > 0 else 0
                percent_predicted = (trail_area_meters / seismic_area) * 100 if seismic_area > 0 else 0
            else:
                trails_area_within_seismic = 0
                trail_area_meters = 0
                percent_vi = 0
                percent_predicted = 0

            # Add the results for the current polygon to the results list
            results_list.append({
                'Polygon_ID': index,
                'Total_Area_VI_Trails': trails_area_within_seismic,  # Total area of buffered VI trails within seismic lines
                'Total_Area_CNN_Trails': trail_area_meters,  # Total area of predicted trails within seismic lines
                'Seismic_Area': seismic_area,  # Total area of seismic lines within the polygon
                'Percent_Area_VI_Trails': percent_vi,  # Percentage of VI trail area relative to seismic area
                'Percent_Area_CNN_Trails': percent_predicted  # Percentage of predicted trail area relative to seismic area
            })

# Convert the list of results to a DataFrame
trail_on_lines = pd.DataFrame(results_list)

# Optionally, save the results to a CSV file
results_csv_path = '/media/irro/Irro/HumanFootprint/AA/trail_length_lines.csv'
trail_on_lines.to_csv(results_csv_path, index=False)

In [ ]:
trail_on_lines[trail_on_lines['Total_Area_CNN_Trails'] > -1].astype(int)

In [ ]:
mean_footprint_area = int(trail_on_lines[trail_on_lines['Seismic_Area'] > 50]['Seismic_Area'].mean().astype(int))
std_footprint_area = int(trail_on_lines[trail_on_lines['Seismic_Area'] > 50]['Seismic_Area'].std().astype(int))

print('Mean area of line footprint:\n', f"{mean_footprint_area} ± {std_footprint_area}")
print('Mean area of line footprint:\n', f"{mean_footprint_area/25} ± {std_footprint_area/25}%")

In [ ]:
mean_trailVI_percent = int(trail_on_lines[trail_on_lines['Seismic_Area'] > 50]['Percent_Area_VI_Trails'].mean().astype(int))
std_trailVI_percent = int(trail_on_lines[trail_on_lines['Seismic_Area'] > 50]['Percent_Area_VI_Trails'].std().astype(int))

print('Mean % of VI trail ON LINES:\n', f"{mean_trailVI_percent} ± {std_trailVI_percent}")

In [ ]:
mean_trailCNN_percent = int(trail_on_lines[trail_on_lines['Seismic_Area'] > 50]['Percent_Area_CNN_Trails'].mean().astype(int))
std_trailCNN_percent = int(trail_on_lines[trail_on_lines['Seismic_Area'] > 50]['Percent_Area_CNN_Trails'].std().astype(int))

print('Mean % of VI trail ON LINES:\n', f"{mean_trailCNN_percent} ± {std_trailCNN_percent}")

### Areas outside of seismic lines

In [ ]:
# Load the data
polygons = gpd.read_file('/media/irro/Irro/HumanFootprint/AA_delineation/polygons.shp')
all_trails = gpd.read_file("/media/irro/Irro/HumanFootprint/AA/all_trails.shp")
seismic_lines = gpd.read_file('/media/irro/Irro/Kirby/FootprintsKirby/Footprint_SiteE.shp')
buffered_trails = gpd.read_file('/media/irro/Irro/HumanFootprint/AA/all_trails_0.5buf.shp')
cnn_map = '/media/irro/Irro/HumanFootprint/cnn_map_test.tif'

# Set CRS for all_trails to match polygons if it's None
if all_trails.crs is None:
    all_trails = all_trails.set_crs(polygons.crs)
    print("CRS set for all_trails to match polygons.")

# Reproject both polygons and all_trails to match seismic_lines CRS
polygons = polygons.to_crs(seismic_lines.crs)
all_trails = all_trails.to_crs(seismic_lines.crs)
buffered_trails = buffered_trails.to_crs(polygons.crs)

# Check CRS
print("CRS for polygons:", polygons.crs)
print("CRS for all_trails:", all_trails.crs)
print("CRS for seismic_lines:", seismic_lines.crs)

In [ ]:
# DataFrame to store the results
results_list_outside = []

# Open the raster file for trail predictions
with rasterio.open(cnn_map) as src:
    affine = src.transform

    # Loop through each polygon
    for index, polygon in polygons.iterrows():
        # Calculate the total area of the polygon
        polygon_area = polygon.geometry.area

        # Intersect seismic lines with the polygon
        seismic_in_polygon = gpd.overlay(seismic_lines, gpd.GeoDataFrame([polygon], columns=['geometry'], crs=polygons.crs), how='intersection')
        
        # Check if there are seismic lines within the polygon
        if not seismic_in_polygon.empty:
            # Calculate the total area of seismic lines within the polygon
            seismic_area = seismic_in_polygon.geometry.unary_union.area

            # Calculate the area outside seismic lines within the polygon
            outside_area = polygon_area - seismic_area

            # Calculate the area of buffered trails outside the seismic lines in the polygon
            trails_out_seismic = gpd.overlay(buffered_trails, seismic_in_polygon, how='difference')
            trails_area_out_seismic = trails_out_seismic.geometry.area.sum() if not trails_out_seismic.empty else 0

            # Calculate the area of predicted trails outside the seismic area
            outside_intersected_area = polygon['geometry'].difference(seismic_in_polygon.unary_union)
            outside_intersected_mask = geometry_mask([outside_intersected_area], transform=src.transform, invert=True, out_shape=src.shape)

            # Extract the masked area from the raster for trails outside seismic area
            outside_masked_raster, _ = rio_mask(src, [outside_intersected_area], crop=True)
            outside_trail_raster = np.where(outside_masked_raster[0] > 10, 1, 0)  # Apply threshold for trail detection

            # Calculate the area of predicted trails within the outside intersected area
            outside_trail_area_pixels = outside_trail_raster.sum()
            outside_trail_area_meters = outside_trail_area_pixels * (src.res[0] * src.res[1])
            print(outside_trail_area_meters)
            print('Out area: ', outside_area)
            
            # Store results for the current polygon
            results_list_outside.append({
                'Polygon_ID': index,
                'Total_Area_VI_Trails_Outside': trails_area_out_seismic,  # Total area of buffered VI trails outside seismic lines
                'Total_Area_CNN_Trails_Outside': outside_trail_area_meters,  # Total area of predicted trails outside seismic lines
                'Outside_Area': outside_area  # Total area outside seismic lines within the polygon
            })
        else:
            # If there are no seismic lines in the polygon, consider the entire polygon as outside area
            outside_area = polygon_area
            trails_out_seismic = buffered_trails[buffered_trails.intersects(polygon.geometry)]
            trails_area_out_seismic = trails_out_seismic.geometry.area.sum()

            outside_trail_area_meters = 0  # No predicted trails since there's no seismic line

            # Store results for the current polygon
            results_list_outside.append({
                'Polygon_ID': index,
                'Total_Area_VI_Trails_Outside': trails_area_out_seismic,
                'Total_Area_CNN_Trails_Outside': outside_trail_area_meters,
                'Outside_Area': outside_area
            })

# Convert the list of results to a DataFrame
trail_outside_lines = pd.DataFrame(results_list_outside)

# Optionally, save the results to a CSV file
results_csv_path_outside = '/media/irro/Irro/HumanFootprint/AA/trail_length_outside_lines.csv'
trail_outside_lines.to_csv(results_csv_path_outside, index=False)


In [ ]:
trail_outside_lines

In [ ]:
trail_outside_lines['CNN_percent'] = trail_outside_lines2.Total_Area_CNN_Trails_Outside *100 / trail_outside_lines2.Outside_Area

mean_trailCNN_percent = int(trail_outside_lines[trail_outside_lines['CNN_percent'] > 5]['CNN_percent'].mean().astype(int))
std_trailCNN_percent = int(trail_outside_lines[trail_outside_lines['CNN_percent'] > 5]['CNN_percent'].std().astype(int))

print('Mean % of CNN trail ON LINES:\n', f"{mean_trailCNN_percent} ± {std_trailCNN_percent}")

##### Stats

In [ ]:
# Filter DataFrame for polygons where predicted trail area is greater than 10
filtered_df = trail_on_lines[trail_on_lines['Total_Area_CNN_Trails'] > 10].astype(int)

# Calculate mean and standard deviation for the filtered DataFrame
mean_values = filtered_df.mean().astype(int)
std_dev_values = filtered_df.std().astype(int)

# Calculate mean and standard deviation for the filtered DataFrame
mean_values = filtered_df.mean().astype(int)
std_dev_values = filtered_df.std().astype(int)

# Print the results
for col in filtered_df.columns[1:]:
    mean = mean_values[col].round(0)
    std_dev = std_dev_values[col].round(0)
    print(f"{col}: {mean} ± {std_dev}")

### Ecosite Type Trail Map

In [4]:
# ecosite_map_path = '/media/irro/Irro/HumanFootprint/cnn_test_classes.tif'
# predicted_trail_map_path = '/media/irro/Irro/HumanFootprint/cnn_test_se.tif'
# reprojected_ecosite_map_path = f'{ecosite_map_path[:-4]}_2956.tif'  # Path to save reprojected ecosite map
# resampled_ecosite_map_path = f'{ecosite_map_path[:-4]}_10cm.tif'  # Path to save resampled ecosite map

In [5]:
ecosite_map_path = '/media/irro/Irro/Kirby/Sentinel/LandCover_ROI_Kirby.tif'
# predicted_trail_map_path = '/media/irro/Irro/Kirby/DTM_10cm_binning_488_6131_496_6137_CNN9ep_512_358max.tif'
predicted_trail_map_path = '/media/irro/Irro/Kirby/DTM_50cm_binning_488_6131_496_6137_CNN9ep_256_179max.tif'

reprojected_ecosite_map_path = f'{ecosite_map_path[:-4]}_2956.tif'  # Path to save reprojected ecosite map
resampled_ecosite_map_path = f'{ecosite_map_path[:-4]}_10cm.tif'  # Path to save resampled ecosite map

##### Reproject to UTM

In [6]:
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling

def reproject_to_utm(src, dst_crs):
    transform, width, height = calculate_default_transform(
        src.crs, dst_crs, src.width, src.height, *src.bounds)
    kwargs = src.meta.copy()
    kwargs.update({
        'crs': dst_crs,
        'transform': transform,
        'width': width,
        'height': height,
        'dtype': rasterio.float32
    })

    reprojected = np.empty(shape=(height, width), dtype=rasterio.float32)

    reproject(
        source=rasterio.band(src, 1),
        destination=reprojected,
        src_transform=src.transform,
        src_crs=src.crs,
        dst_transform=transform,
        dst_crs=dst_crs,
        resampling=Resampling.nearest)

    return reprojected, kwargs

# Reproject ecosite raster to UTM (EPSG:2956)
with rasterio.open(ecosite_map_path) as ecosite_src:
    ecosite_data, ecosite_meta = reproject_to_utm(ecosite_src, 'EPSG:2956')

    # Save the reprojected raster
    with rasterio.open(reprojected_ecosite_map_path, 'w', **ecosite_meta) as dst:
        dst.write(ecosite_data, 1)

    print(f"Saved reprojected ecosite raster to {reprojected_ecosite_map_path}")

Saved reprojected ecosite raster to /media/irro/Irro/Kirby/Sentinel/LandCover_ROI_Kirby_2956.tif


##### Resample

In [8]:
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling

def resample_raster_to_match_resolution(ecosite_map_path, cnn_map_path, output_path):
    with rasterio.open(cnn_map_path) as cnn_src:
        cnn_transform, cnn_width, cnn_height = cnn_src.transform, cnn_src.width, cnn_src.height

    with rasterio.open(ecosite_map_path) as ecosite_src:
        # Calculate the transformation to match the CNN map resolution
        transform, width, height = calculate_default_transform(
            ecosite_src.crs, ecosite_src.crs, cnn_width, cnn_height, *cnn_src.bounds)
        kwargs = ecosite_src.meta.copy()
        kwargs.update({
            'transform': transform,
            'width': width,
            'height': height,
            'dtype': 'float32'
        })

        # Perform the resampling
        with rasterio.open(output_path, 'w', **kwargs) as dst:
            for i in range(1, ecosite_src.count + 1):
                reproject(
                    source=rasterio.band(ecosite_src, i),
                    destination=rasterio.band(dst, i),
                    src_transform=ecosite_src.transform,
                    src_crs=ecosite_src.crs,
                    dst_transform=transform,
                    dst_crs=ecosite_src.crs,
                    resampling=Resampling.nearest)

# Resample and save the ecosite raster
resample_raster_to_match_resolution(reprojected_ecosite_map_path, predicted_trail_map_path, resampled_ecosite_map_path)

##### Area Calc

In [9]:
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling
import numpy as np
import pandas as pd
from rasterio.windows import from_bounds

def process_chunk(ecosite_chunk, trail_chunk, pixel_size, target_classes):
    chunk_results = []
#     unique_classes, counts = np.unique(ecosite_chunk, return_counts=True)
#     class_distribution = dict(zip(unique_classes, counts))
#     print(f"Unique classes in chunk: {class_distribution}")

    ecosite_chunk[np.isin(ecosite_chunk, [1, 9])] = 10
    trail_chunk = np.where(trail_chunk > 10, 1, 0)

    for ecosite_class in target_classes:
        ecosite_mask = ecosite_chunk == ecosite_class
        trail_mask = ecosite_mask & trail_chunk.astype(bool)

        ecosite_area_pixels = np.sum(ecosite_mask)
        trail_area_pixels = np.sum(trail_mask)

        ecosite_area_meters = ecosite_area_pixels * pixel_size * pixel_size
        trail_area_meters = trail_area_pixels * pixel_size * pixel_size

        trail_percentage = (trail_area_meters / ecosite_area_meters) * 100 if ecosite_area_meters > 0 else 0

        chunk_results.append((ecosite_class, trail_area_meters, ecosite_area_meters, trail_percentage))

#         print(f"Class {ecosite_class}: Ecosite Area Pixels: {ecosite_area_pixels}, Trail Area Pixels: {trail_area_pixels}")

    return chunk_results

def get_square_chunks(width, height, chunk_size):
    # Generates square (or rectangular) chunks of specified size
    for i in range(0, height, chunk_size):
        for j in range(0, width, chunk_size):
            yield Window(j, i, min(chunk_size, width - j), min(chunk_size, height - i))

In [11]:
from rasterio.windows import Window

results_df = pd.DataFrame(columns=['Ecosite_Class', 'Trail_Area', 'Ecosite_Area', 'Trail_Percentage'])           
chunk_size = 200  # Define your desired chunk size
target_classes = [0, 1, 2, 6, 9]  # Your target classes

# Processing
with rasterio.open(resampled_ecosite_map_path) as ecosite_src, rasterio.open(predicted_trail_map_path) as trail_src:
    for window in get_square_chunks(ecosite_src.width, ecosite_src.height, chunk_size):
        ecosite_chunk = ecosite_src.read(1, window=window)
        trail_chunk = trail_src.read(1, window=window)

        chunk_results = process_chunk(ecosite_chunk, trail_chunk, 0.1, target_classes)  # 0.1m is the pixel size for the trail raster

        # Collect results and concatenate with the main DataFrame
        chunk_results_df = [pd.DataFrame([{
            'Ecosite_Class': result[0],
            'Trail_Area': result[1],
            'Ecosite_Area': result[2],
            'Trail_Percentage': result[3]
        }]) for result in chunk_results]
        results_df = pd.concat([results_df, *chunk_results_df], ignore_index=True)

# Calculate final results
final_results_df = results_df.groupby('Ecosite_Class').sum().astype(int)
final_results_df['Trail_Percentage'] = (final_results_df['Trail_Area'] / final_results_df['Ecosite_Area']) * 100
print(final_results_df)

               Trail_Area  Ecosite_Area  Trail_Percentage
Ecosite_Class                                            
0                   29653        402330          7.370318
1                       0             0               NaN
2                   61444        399564         15.377762
6                   81607        911309          8.954921
9                       0             0               NaN


# Random points for AA

In [ ]:
'''The code first reads the weak, LiDAR, and interpolated trail datasets, merges them into a single GeoDataFrame, 
and saves this as a new file.'''

### Reasing strong, weak, lidar, and interpolated trails
strong_trails = gpd.read_file('/media/irro/Irro/HumanFootprint/AA_delineation/XC_trails_strong.gpkg')
weak_trails = gpd.read_file('/media/irro/Irro/HumanFootprint/AA_delineation/all_weak_trails.gpkg')

#### AOIs
polygons = gpd.read_file('/media/irro/Irro/HumanFootprint/AA_delineation/polygons.shp')
polygons = polygons.reset_index()

In [ ]:
import geopandas as gpd
import random
from shapely.geometry import Point

def generate_random_points_on_line(line, num_points):
    if line.is_empty or line is None:
        return []

    points = []
    line_length = line.length
    for _ in range(num_points):
        distance = random.uniform(0, line_length)
        random_point = line.interpolate(distance)
        points.append(random_point)
    return points

# Create an empty GeoDataFrame for the random points
crs = polygons.crs
# Standardize CRS across all datasets
strong_trails = strong_trails.to_crs(crs)
weak_trails = weak_trails.to_crs(crs)

random_points_gdf = gpd.GeoDataFrame(columns=['geometry', 'trail_type', 'polygon_id'], crs=crs)
number_of_points = 1

# Create an empty list to store GeoDataFrames
gdfs = []

# Generate random points for each polygon
for index, polygon in polygons.iterrows():
    polygon_id = polygon['index']  # Adjust the 'ID' to the appropriate identifier column
    polygon_gdf = gpd.GeoDataFrame([polygon], columns=['geometry'], crs=crs)
    
    # Intersect the trails with the polygon
    for trail_gdf, trail_type in [(strong_trails, 'strong'), (weak_trails, 'weak')]:
        intersected_trails = gpd.overlay(trail_gdf, polygon_gdf, how='intersection')
#         print(f"Polygon ID {polygon_id}: {len(intersected_trails)} intersecting {trail_type} trails")

        if not intersected_trails.empty:
            for _, trail_segment in intersected_trails.iterrows():
                if not trail_segment.geometry.is_empty and trail_segment.geometry is not None:
                    points = generate_random_points_on_line(trail_segment.geometry, number_of_points)
                    points_gdf = gpd.GeoDataFrame({'geometry': points, 'trail_type': trail_type, 'polygon_id': polygon_id}, crs=crs)
                    gdfs.append(points_gdf)

# Concatenate all GeoDataFrames
random_points_gdf = pd.concat(gdfs, ignore_index=True)

# Save to a shapefile
random_points_path = "/media/irro/Irro/HumanFootprint/AA/random_points.shp"
random_points_gdf.to_file(random_points_path, driver='ESRI Shapefile')

In [ ]:
random_points_path = "/media/irro/Irro/HumanFootprint/AA/random_points.shp"
random_points = gpd.read_file(random_points_path)

In [ ]:
random_points.groupby('trail_type').count()

In [ ]:
len(random_points)

### Non-trail points

In [ ]:
import geopandas as gpd
import random
from shapely.geometry import Point

def generate_random_points_outside_trails(polygons, trails, num_points_per_polygon, buffer_dist):
    print('Requested number of points per polygon:', num_points_per_polygon)
    # Ensure trails is a GeoDataFrame
    if isinstance(trails, gpd.GeoSeries):
        trails = gpd.GeoDataFrame(geometry=trails)

    # Set the CRS of buffered_trails to match polygons if it's not set
    if trails.crs != polygons.crs:
        trails = trails.set_crs(polygons.crs)

    # Buffer the trails
    buffered_trails = trails.copy()
    buffered_trails['geometry'] = trails.buffer(buffer_dist)

    # Generate random points
    all_points = []
    for _, polygon in polygons.iterrows():
        print(_)
        polygon_id = polygon['index']  # Adjust this if 'index' is not the correct identifier
        # Calculate the difference
        non_trail_area = gpd.overlay(gpd.GeoDataFrame([polygon], columns=['geometry'], crs=polygons.crs), 
                                     buffered_trails, 
                                     how='difference')

        points = []
        for _, area in non_trail_area.iterrows():
            minx, miny, maxx, maxy = area.geometry.bounds
            while len(points) < num_points_per_polygon:
                x = random.uniform(minx, maxx)
                y = random.uniform(miny, maxy)
                point = Point(x, y)
                if area.geometry.contains(point):
                    points.append(point)

        # Create a GeoDataFrame for points in this polygon
        polygon_points_gdf = gpd.GeoDataFrame({'geometry': points, 'polygon_id': polygon_id}, crs=polygons.crs)
        all_points.append(polygon_points_gdf)

    # Combine points from all polygons
    combined_points_gdf = pd.concat(all_points, ignore_index=True)
    return combined_points_gdf

# Example usage
all_trails = gpd.read_file("/media/irro/Irro/HumanFootprint/AA/all_trails.shp", driver='ESRI Shapefile')

# Generate non-trail points
num_points_per_polygon = 300  # Adjust this number as needed
non_trail_points = generate_random_points_outside_trails(polygons, all_trails, num_points_per_polygon, 1)  # 1 meter buffer

print('Generated non-trail points:', len(non_trail_points))
points_shp = "/media/irro/Irro/HumanFootprint/AA/random_points.shp"

# Save to shapefile
non_trail_points_shp = points_shp.replace('.shp', '_non_trail_points.shp')
non_trail_points.to_file(non_trail_points_shp)

In [ ]:
points_shp = "/media/irro/Irro/HumanFootprint/AA/random_points.shp"
non_trail_points_shp = points_shp.replace('.shp', '_non_trail_points.shp')
non_trail_points = gpd.read_file(non_trail_points_shp)

In [ ]:
print(len(non_trail_points))

##### Combine all points

In [ ]:
# Load the shapefiles
points_shp = "/media/irro/Irro/HumanFootprint/AA/random_points.shp"
# nontrails_shp = '/media/irro/Irro/HumanFootprint/AA/random_points_non_trail_points.shp'

trail_points_gdf = gpd.read_file(points_shp)

# Add 'trail_type' column to non-trail points and set it to 'no_trails'
non_trail_points['trail_type'] = 'no_trails'

# Combine the two GeoDataFrames
random_points_gdf = pd.concat([trail_points_gdf, non_trail_points], ignore_index=True)

# Save the combined GeoDataFrame to a new shapefile (optional)
# combined_shp = "/media/irro/Irro/HumanFootprint/AA/combined_trail_and_non_trail_points.shp"
# random_points_gdf.to_file(combined_shp)

In [ ]:
len(random_points_gdf)

# AA

In [ ]:
import geopandas as gpd
import rasterio
import numpy as np
import pandas as pd
import math
from shapely.geometry import Point
from rasterio.features import geometry_mask

# Paths to the raster and points shapefile
cnn_map = '/media/irro/Irro/HumanFootprint/AA/cnn_map_test3.tif'### DSM10cmbest
# cnn_map = '/media/irro/Irro/HumanFootprint/AA/cnn_map_test.tif' ### DTM10cmbest

# cnn_map = '/media/irro/Irro/HumanFootprint/AA/DTM_50cm_binning_488_6131_496_6137_CNN9ep_256_179max.tif' ### Final DTM50cm
# cnn_map = '/media/irro/Irro/HumanFootprint/AA/DTM_50cm_binning_488_6131_496_6137_CNNover50e_256_200.tif' ### First DTM50cm

# cnn_map = '/media/irro/Irro/HumanFootprint/AA/Kirby_10cm_normDTM6m_fen_Human_DTM_512_byCNN_9ep_good_512_358max.tif'
# cnn_map = '/media/irro/Irro/HumanFootprint/AA/PFT_KirbySouth_July2022_DTM_Human_DTM_512_byCNN_9ep_good_512_358max.tif'
# cnn_map = '/media/irro/Irro/HumanFootprint/AA/DTM_10cm_newCNN9ep_512_358max.tif'

points_shp = "/media/irro/Irro/HumanFootprint/AA/combined_trail_and_non_trail_points.shp"

# Load the points
random_points_gdf = gpd.read_file(points_shp)

p = 1  # Threshold for the raster data

# Parameters for the analysis
buffer_radius = 1  # Buffer radius in meters
area_thre = 0.2  # Area threshold for considering a true positive

# Optionally, save the results DataFrame to a CSV file
output_csv_path = cnn_map.replace('.tif', f'_point_analysis_prob{p}.csv')
output_shp_path = cnn_map.replace('.tif', f'_prob{p}_AA.shp')

# Initialize a DataFrame to store the results
result = []
# for p in [5,10,15,20,25,30,35,40,45,50]:
# Open the raster file
with rasterio.open(cnn_map) as src:
    affine_transform = src.transform
    cell_size = src.res[0]

    # Loop through each point
    for index, row in random_points_gdf.iterrows():
        point = row['geometry']
        polygon_id = row['polygon_id']
        trail_type = row['trail_type']

        # Buffer the point
        buffered_point = point.buffer(buffer_radius)

        # Determine the bounding box for the buffered point and read the relevant raster portion
        minx, miny, maxx, maxy = buffered_point.bounds
        window = rasterio.windows.from_bounds(minx, miny, maxx, maxy, transform=affine_transform)
        raster_subset = src.read(1, window=window)

        # Create a binary map based on the threshold
        arr = np.where(raster_subset < p, 0, 1)

        # Calculate the percentage of the buffer area that is covered by the trail
        trail_pixel_count = np.sum(arr)

        # Adjust buffer area calculation to consider a square enclosing the buffer circle
        side_length = buffer_radius * 2  # Side length of the square enclosing the buffer circle
        buffer_area_pixel_count = (side_length / cell_size) ** 2  # Area of the square in pixels

        trail_area_percentage = trail_pixel_count / buffer_area_pixel_count

        # Determine TP, FP, TN, FN based on the trail area percentage and trail type
        is_trail_predicted = trail_area_percentage > area_thre
        is_true_positive = is_trail_predicted and trail_type != 'no_trails'
        is_false_positive = is_trail_predicted and trail_type == 'no_trails'
        is_true_negative = not is_trail_predicted and trail_type == 'no_trails'
        is_false_negative = not is_trail_predicted and trail_type != 'no_trails'

        # Store results for each point
        result.append({
            'Point_Index': index,
            'Polygon_ID': polygon_id,
            'Trail_Type': trail_type,
            'Trail_Area_Percentage': trail_area_percentage,
            'Is_Trail': is_trail_predicted,
            'Is_TP': is_true_positive,
            'Is_FP': is_false_positive,
            'Is_TN': is_true_negative,
            'Is_FN': is_false_negative,
            'geometry': point
        })

        # Print the results for the current point
        print(f"Point {index}, Polygon {polygon_id}, Trail type {trail_type}: Trail Area % = {trail_area_percentage:.2f}, TP = {is_true_positive}, FP = {is_false_positive}, TN = {is_true_negative}, FN = {is_false_negative}")

In [ ]:
# Convert the results to a DataFrame
results = pd.DataFrame(result)
results.to_csv(output_csv_path, index=False)
print(f'N of points: {len(results)}')
print('Saved to: ', output_csv_path)

# Apply the function to create a new column
results['Category'] = results.apply(categorize, axis=1)

# Convert the DataFrame to a GeoDataFrame
df = gpd.GeoDataFrame(results, geometry='geometry')
df.crs = "EPSG:2956"  # Replace with the correct EPSG code
df.to_file(output_shp_path, driver='ESRI Shapefile')

print(f"Shapefile saved to {output_shp_path}")

# Filter out 'weak' trails
strong_non_trails_df = results[results['Trail_Type'] != 'weak']

# Initialize counters
tp_strong = sum((strong_non_trails_df['Trail_Type'] == 'strong') & strong_non_trails_df['Is_TP'])
fn_strong = sum((strong_non_trails_df['Trail_Type'] == 'strong') & strong_non_trails_df['Is_FN'])
fp_strong = sum((strong_non_trails_df['Trail_Type'] == 'no_trails') & strong_non_trails_df['Is_FP'])
tn_strong = sum((strong_non_trails_df['Trail_Type'] == 'no_trails') & strong_non_trails_df['Is_TN'])

# Calculate precision, recall, and F1-score for strong vs. non-trails
precision_strong = tp_strong / (tp_strong + fp_strong) if tp_strong + fp_strong > 0 else 0
recall_strong = tp_strong / (tp_strong + fn_strong) if tp_strong + fn_strong > 0 else 0
f1_score_strong = 2 * (precision_strong * recall_strong) / (precision_strong + recall_strong) if precision_strong + recall_strong > 0 else 0

# Print results
print("Confusion Matrix - STRONG vs. Non-Trails:")
print(f"TP: {tp_strong}, FN: {fn_strong}, FP: {fp_strong}, TN: {tn_strong}")
print(f"Precision: {precision_strong:.2f}, Recall: {recall_strong:.2f}, F1-Score: {f1_score_strong:.2f}")

# Filter out only non-trails
all_trails_df = results[results['Trail_Type'] == 'weak']

# Initialize counters
tp_all = sum(all_trails_df['Is_TP'])
fn_all = sum(all_trails_df['Is_FN'])
fp_all = sum((results['Trail_Type'] == 'no_trails') & results['Is_FP'])
tn_all = sum((results['Trail_Type'] == 'no_trails') & results['Is_TN'])

# Calculate precision, recall, and F1-score for all trails vs. non-trails
precision_all = tp_all / (tp_all + fp_all) if tp_all + fp_all > 0 else 0
recall_all = tp_all / (tp_all + fn_all) if tp_all + fn_all > 0 else 0
f1_score_all = 2 * (precision_all * recall_all) / (precision_all + recall_all) if precision_all + recall_all > 0 else 0

# Print results
print("\nConfusion Matrix - WEAK Trails vs. Non-Trails:")
print(f"TP: {tp_all}, FN: {fn_all}, FP: {fp_all}, TN: {tn_all}")
print(f"Precision: {precision_all:.2f}, Recall: {recall_all:.2f}, F1-Score: {f1_score_all:.2f}")

##### Confusion Matrix

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

# Confusion Matrix data for Strong vs. Non-Trails
cm_strong = [[tp_all, fn_all], [fp_all, tn_all]]
cm_strong_df = pd.DataFrame(cm_strong, index=['Trails', 'Non-Trails'], columns=['Trails', 'Non-Trails'])

# Plotting the Strong vs. Non-Trails confusion matrix
plt.figure(figsize=(4,4))
sns.heatmap(cm_strong_df, annot=True, cmap='Blues', fmt='g')
plt.title(f'{os.path.basename(cnn_map)[:-4]}')
plt.xlabel('Actual')
plt.ylabel('Predicted')
plt.show()


### Across Poly

In [ ]:
def calculate_metrics(tp, fp, fn):
    """Helper function to calculate precision, recall, and F1-score."""
    precision = tp / (tp + fp) if tp + fp > 0 else 0
    recall = tp / (tp + fn) if tp + fn > 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if precision + recall > 0 else 0
    return precision, recall, f1_score

# Initialize a list to store metrics for each polygon and trail type
metrics = []

for poly in range(0, 11):
    subset = results[results.Polygon_ID == poly]

    # For Strong Trails
    strong_trails_df = subset[subset['Trail_Type'] == 'strong']
    tp_strong = sum(strong_trails_df['Is_TP'])
    fn_strong = sum(strong_trails_df['Is_FN'])
    fp_strong = sum((subset['Trail_Type'] == 'no_trails') & subset['Is_FP'])
    # Metrics for Strong Trails
    precision_strong, recall_strong, f1_score_strong = calculate_metrics(tp_strong, fp_strong, fn_strong)
    metrics.append({
        'Polygon_ID': poly,
        'Trail_Type': 'Strong Trails',
        'Precision': round(precision_strong, 2),
        'Recall': round(recall_strong, 2),
        'F1-Score': round(f1_score_strong, 2)
    })

    # For All Trails (Strong and Weak)
    all_trails_df = subset[subset['Trail_Type'].isin(['weak'])]
    tp_all = sum(all_trails_df['Is_TP'])
    fn_all = sum(all_trails_df['Is_FN'])
    fp_all = sum((subset['Trail_Type'] == 'no_trails') & subset['Is_FP'])
    # Metrics for All Trails
    precision_all, recall_all, f1_score_all = calculate_metrics(tp_all, fp_all, fn_all)
    metrics.append({
        'Polygon_ID': poly,
        'Trail_Type': 'Weak Trails',
        'Precision': round(precision_all, 2),
        'Recall': round(recall_all, 2),
        'F1-Score': round(f1_score_all, 2)
    })

# Convert the list to a DataFrame
metrics = pd.DataFrame(metrics)

# Optionally, save the DataFrame to a CSV file
output_csv_path = f'/media/irro/Irro/HumanFootprint/AA/{os.path.basename(cnn_map)[:-4]}_{p}.csv'
metrics.to_csv(output_csv_path, index=False)

# Print the final DataFrame
print(metrics[metrics.Trail_Type == 'Weak Trails'])

In [ ]:
### STRONG TRAILS

# Select only the rows for 'Strong Trails'
strong_trails_metrics = metrics[metrics['Trail_Type'] == 'Strong Trails']

# Calculate mean and std for Precision, Recall, and F1-Score
mean_precision = strong_trails_metrics['Precision'].mean()
std_precision = strong_trails_metrics['Precision'].std()

mean_recall = strong_trails_metrics['Recall'].mean()
std_recall = strong_trails_metrics['Recall'].std()

mean_f1_score = strong_trails_metrics['F1-Score'].mean()
std_f1_score = strong_trails_metrics['F1-Score'].std()

# Print the results
print("Mean and Standard Deviation for Strong Trails:")
print(f"Precision: Mean = {mean_precision:.2f}, Std = {std_precision:.2f}")
print(f"Recall: Mean = {mean_recall:.2f}, Std = {std_recall:.2f}")
print(f"F1-Score: Mean = {mean_f1_score:.2f}, Std = {std_f1_score:.2f}")


In [ ]:
### WEAK TRAILS

strong_trails_metrics = metrics[metrics['Trail_Type'] == 'Weak Trails']

# Calculate mean and std for Precision, Recall, and F1-Score
mean_precision = strong_trails_metrics['Precision'].mean()
std_precision = strong_trails_metrics['Precision'].std()

mean_recall = strong_trails_metrics['Recall'].mean()
std_recall = strong_trails_metrics['Recall'].std()

mean_f1_score = strong_trails_metrics['F1-Score'].mean()
std_f1_score = strong_trails_metrics['F1-Score'].std()

# Print the results
print("Mean and Standard Deviation for Weak Trails:")
print(f"Precision: Mean = {mean_precision:.2f}, Std = {std_precision:.2f}")
print(f"Recall: Mean = {mean_recall:.2f}, Std = {std_recall:.2f}")
print(f"F1-Score: Mean = {mean_f1_score:.2f}, Std = {std_f1_score:.2f}")


### PreRec

In [ ]:
points_shp = "/media/irro/Irro/HumanFootprint/AA/combined_trail_and_non_trail_points.shp"

# Load the points
random_points_gdf = gpd.read_file(points_shp)

# Parameters for the analysis
buffer_radius = 1  # Buffer radius in meters
area_thre = 0.2  # Area threshold for considering a true positive

# Initialize a DataFrame to store the results
results = []

p_list = [1, 3, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85, 90, 95, 99]
for p in p_list:
    print('Prob: ', p)
    # Optionally, save the results DataFrame to a CSV file
    output_csv_path = cnn_map.replace('.tif', f'_point_analysis_prob{p}.csv')
    output_shp_path = cnn_map.replace('.tif', f'_prob{p}_AA.shp')

# Open the raster file
    with rasterio.open(cnn_map) as src:
        affine_transform = src.transform
        cell_size = src.res[0]

        # Loop through each point
        for index, row in random_points_gdf.iterrows():
            point = row['geometry']
            polygon_id = row['polygon_id']
            trail_type = row['trail_type']

            # Buffer the point
            buffered_point = point.buffer(buffer_radius)

            # Determine the bounding box for the buffered point and read the relevant raster portion
            minx, miny, maxx, maxy = buffered_point.bounds
            window = rasterio.windows.from_bounds(minx, miny, maxx, maxy, transform=affine_transform)
            raster_subset = src.read(1, window=window)

            # Create a binary map based on the threshold
            arr = np.where(raster_subset < p, 0, 1)

            # Calculate the percentage of the buffer area that is covered by the trail
            trail_pixel_count = np.sum(arr)

            # Adjust buffer area calculation to consider a square enclosing the buffer circle
            side_length = buffer_radius * 2  # Side length of the square enclosing the buffer circle
            buffer_area_pixel_count = (side_length / cell_size) ** 2  # Area of the square in pixels

            trail_area_percentage = trail_pixel_count / buffer_area_pixel_count

            # Determine TP, FP, TN, FN based on the trail area percentage and trail type
            is_trail_predicted = trail_area_percentage > area_thre
            is_true_positive = is_trail_predicted and trail_type != 'no_trails'
            is_false_positive = is_trail_predicted and trail_type == 'no_trails'
            is_true_negative = not is_trail_predicted and trail_type == 'no_trails'
            is_false_negative = not is_trail_predicted and trail_type != 'no_trails'

            # Store results for each point
            results.append({
                'Point_Index': index,
                'Polygon_ID': polygon_id,
                'Trail_Type': trail_type,
                'Trail_Area_Percentage': trail_area_percentage,
                'Is_Trail': is_trail_predicted,
                'Is_TP': is_true_positive,
                'Is_FP': is_false_positive,
                'Is_TN': is_true_negative,
                'Is_FN': is_false_negative,
                'geometry': point,
                'prob': p
            })

            # Print the results for the current point
#             print(f"Point {index}, Polygon {polygon_id}, Trail type {trail_type}: Trail Area % = {trail_area_percentage:.2f}, TP = {is_true_positive}, FP = {is_false_positive}, TN = {is_true_negative}, FN = {is_false_negative}")

#     # Convert the results to a DataFrame
    results_df = pd.DataFrame(results)
    results_df.to_csv(output_csv_path, index=False)

print('Saved to: ', output_csv_path)

### Plotting

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Initialize a DataFrame to store precision and recall for different p values
metrics_data = {
    'p': [],
    'Precision_All': [],
    'Recall_All': [],
    'F1_Score_All': [],
    'type': []
}
for trail_type in ['strong', 'weak']:
    for p in p_list:
    #     print('Probability: ', p)
        subset = results_df[results_df['prob'] == p]

        # Filter out only non-trails
        all_trails_df = subset[subset['Trail_Type'] == trail_type]

        # Initialize counters
        tp_all = sum(all_trails_df['Is_TP'])
        fn_all = sum(all_trails_df['Is_FN'])
        fp_all = sum((subset['Trail_Type'] == 'no_trails') & subset['Is_FP'])
        tn_all = sum((subset['Trail_Type'] == 'no_trails') & subset['Is_TN'])

        # Calculate precision, recall,b and F1-score for all trails vs. non-trails
        precision_all = tp_all / (tp_all + fp_all) if tp_all + fp_all > 0 else 0
        recall_all = tp_all / (tp_all + fn_all) if tp_all + fn_all > 0 else 0
        f1_score_all = 2 * (precision_all * recall_all) / (precision_all + recall_all) if precision_all + recall_all > 0 else 0

    #     print(f"Precision: {precision_all:.2f}, Recall: {recall_all:.2f}, F1-Score: {f1_score_all:.2f}")

        metrics_data['p'].append(p)
        metrics_data['Precision_All'].append(precision_all)
        metrics_data['Recall_All'].append(recall_all)
        metrics_data['F1_Score_All'].append(f1_score_all)
        metrics_data['type'].append(trail_type)

# Convert the dictionary to a DataFrame
metrics_df = pd.DataFrame(metrics_data)

In [ ]:
metrics_df[metrics_df.type == 'strong'].head(10)

##### AP

In [ ]:
from sklearn.metrics import auc

# Sorting the DataFrame by recall values
df = metrics_df[metrics_df.type == 'strong']
df.sort_values('Recall_All', inplace=True)

# Calculating the area under the precision-recall curve
ap = auc(df['Recall_All'], df['Precision_All'])

# Plotting Precision-Recall curve
plt.figure(figsize=(4, 4))
plt.plot(df['Recall_All'], df['Precision_All'], marker='o')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title(f'{os.path.basename(cnn_map)}')
plt.show()

print(f"Average Precision (AP). Strong Trails: {round(ap,2)}")

In [ ]:
from sklearn.metrics import auc

# Sorting the DataFrame by recall values
df = metrics_df[metrics_df.type == 'weak']
df.sort_values('Recall_All', inplace=True)

# Calculating the area under the precision-recall curve
ap = auc(df['Recall_All'], df['Precision_All'])

# Plotting Precision-Recall curve
plt.figure(figsize=(4, 4))
plt.plot(df['Recall_All'], df['Precision_All'], marker='o')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title(f'{os.path.basename(cnn_map)}')
plt.show()

print(f"Average Precision (AP). Weak Trails: {round(ap,2)}")

### PreRec Curve

In [ ]:
# Filter the DataFrame for strong and weak trails
strong_metrics_df = metrics_df[metrics_df['type'] == 'strong']
weak_metrics_df = metrics_df[metrics_df['type'] == 'weak']

# For p = 100, set precision to the last calculated precision value
strong_last_precision = strong_metrics_df['Precision_All'].iloc[-1]  # second last, as the last is the manually added 100% precision
weak_last_precision = weak_metrics_df['Precision_All'].iloc[-1]

# For p = 0, set recall to the maximum recall value
strong_max_recall = strong_metrics_df['Recall_All'].max()
weak_max_recall = weak_metrics_df['Recall_All'].max()

# Add rows for p = 100 and p = 0
strong_100_row = {'p': 100, 'Precision_All': strong_last_precision, 'Recall_All': 0, 'F1_Score_All': None, 'type': 'strong'}
weak_100_row = {'p': 100, 'Precision_All': weak_last_precision, 'Recall_All': 0, 'F1_Score_All': None, 'type': 'weak'}
strong_0_row = {'p': 0, 'Precision_All': 0, 'Recall_All': strong_max_recall, 'F1_Score_All': None, 'type': 'strong'}
weak_0_row = {'p': 0, 'Precision_All': 0, 'Recall_All': weak_max_recall, 'F1_Score_All': None, 'type': 'weak'}

# Create a new DataFrame by appending extra rows to metrics_df
metrics_df_plot = metrics_df.append([strong_100_row, weak_100_row, strong_0_row, weak_0_row], ignore_index=True)

# Filter the new DataFrame for strong and weak trails, and sort by 'p'
strong_metrics_df = metrics_df_plot[metrics_df_plot['type'] == 'strong'].sort_values(by='p', ascending=True)
weak_metrics_df = metrics_df_plot[metrics_df_plot['type'] == 'weak'].sort_values(by='p', ascending=True)

In [ ]:
# Plotting the Precision-Recall curves
plt.figure(figsize=(5, 5))

# Plot for Strong Trails
plt.plot(strong_metrics_df['Recall_All'], strong_metrics_df['Precision_All'], label='Strong Trails', marker='o', color='blue', linestyle='-', linewidth=2)

# Plot for Weak Trails
plt.plot(weak_metrics_df['Recall_All'], weak_metrics_df['Precision_All'], label='Weak Trails', marker='x', color='red', linestyle='-', linewidth=2)


# Axis labels and title
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title(f'{os.path.basename(cnn_map)[:-34]}')
plt.legend()
plt.grid(True)
plt.xlim(0, 1)  # Extend x-axis to 1
plt.ylim(0, 1)  # Extend y-axis to 1
plt.show()

### AP

In [ ]:
ap = auc(strong_metrics_df['Recall_All'], strong_metrics_df['Precision_All'])
ap

In [ ]:
ap = auc(weak_metrics_df['Recall_All'], weak_metrics_df['Precision_All'])
ap